In [1]:
import pandas as pd

In [2]:
# Variables de referencia
referencia_file_id = '1lrIZQgs81fd4PjNM3FoWDlqb7WqZdZlLc11s6Ne9vT4'
diccionario_gid = '1415696716'
estaciones_gid = '150907099'
opciones_gid = '1046348352'

In [3]:
# Leer archivo y guardarlo
url_diccionario = f'https://docs.google.com/spreadsheets/d/{referencia_file_id}/export?format=csv&gid={diccionario_gid}'
df_diccionario = pd.read_csv(url_diccionario)
indices_columnas_eliminar = df_diccionario['index'][df_diccionario['eliminar'] == 'Sí'].tolist()
columnas = df_diccionario['columna'].tolist()
columnas_nuevo_nombre = df_diccionario['key'].tolist()

In [4]:
# Cargar parámetros de estaciones

url_estaciones = f'https://docs.google.com/spreadsheets/d/{referencia_file_id}/export?format=csv&gid={estaciones_gid}'
df_estaciones = pd.read_csv(url_estaciones)
df_estaciones.head()

,Estación,Nombre estación,Nodo,Troncal
0,CARRERA 47,Carrera 47,3,Calle 80
1,CARRERA 53,Carrera 53,3,Calle 80
2,CDS CARRERA 32,CDS Carrera 32,1,Américas
3,COMAPAN CRA. 43,Comapan Cra. 43,1,Américas
4,ESCUELA MILITAR,Escuela Militar,3,Calle 80


In [5]:
# Cargar parámetros generales
url_opciones = f'https://docs.google.com/spreadsheets/d/{referencia_file_id}/export?format=csv&gid={opciones_gid}'
df_opciones = pd.read_csv(url_opciones)

df_emociones = df_opciones[df_opciones['variable'] == 'Emoción']

df_niveles_confianza = df_opciones[df_opciones['variable'] == 'Confianza']

In [6]:
df_niveles_confianza.head()

,variable,valor,grupo
6,Confianza,1,1. Muy baja
7,Confianza,2,1. Muy baja
8,Confianza,3,2. Baja
9,Confianza,4,2. Baja
10,Confianza,5,2. Baja


In [7]:
# Leer la encuesta
df_encuestas = pd.read_excel('Orgullo_y_apropiacion_de_Transmilenio_2025_Depurada.xlsx', sheet_name='Orgullo y apropiacion de Tra...')

In [8]:
# renombrar columnas segun nuevo nombre 
df_encuestas.columns = columnas_nuevo_nombre

In [9]:
# Eliminar colulmnas no deseadas por índice
df_encuestas = df_encuestas.drop(df_encuestas.columns[indices_columnas_eliminar], axis=1)

In [10]:
# Modificar el formato de la columna "Fecha", que está en formato numérico de Excel, a formato de fecha legible
df_encuestas['fecha'] = pd.to_datetime(df_encuestas['fecha'])

# Agregar la columna "semana" que indica la semana del año correspondiente a la fecha de la encuesta
df_encuestas['semana'] = df_encuestas['fecha'].dt.isocalendar().week

In [11]:
# Agregar las columnas "Nombre estación" y "Troncal" a la base de encuestas, asociadas por "estacion" y "Estación"
df_encuestas = df_encuestas.merge(df_estaciones[['Estación', 'Nombre estación', 'Troncal']], how='left', left_on='estacion', right_on='Estación')
df_encuestas = df_encuestas.drop(columns=['Estación'])

In [12]:
# Agregar la columna df_niveles_confianza['grupo'] como "p1_confianza_demas" según la columna "p01_escala_confianza"
df_encuestas['p01_escala_confianza'] = df_encuestas['p01_escala_confianza'].astype(str)
df_niveles_confianza['valor'] = df_niveles_confianza['valor'].astype(str)

df_encuestas = df_encuestas.merge(
    df_niveles_confianza[['valor', 'grupo']],
    how='left',
    left_on='p01_escala_confianza',
    right_on='valor'
)
df_encuestas = df_encuestas.rename(columns={'grupo': 'p01_confianza_demas_categorias'})
df_encuestas = df_encuestas.drop(columns=['valor'])

C:\Users\User\AppData\Local\Temp\ipykernel_14224\2700727907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_niveles_confianza['valor'] = df_niveles_confianza['valor'].astype(str)


In [13]:
with pd.ExcelWriter('i137_convenio_tm_2025_datos.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_encuestas.to_excel(writer, sheet_name='encuestas', index=False)